# Objectives
This section covers the OpenStack Nova Compute Service. Nova is one of the most complex services in the OpenStack architecture. 

## Agenda
1. Introduction to Compute Engine.
2. Virtual Server Instances (VSI) 
3. Configuring and managing Nova from Horizon
4. Configuring and managing Nova from the CLI
5. Brief introduction to Instance Scheduling

## COA Exam Requirements (15%)
1. __Fundamentals__ 
    * Manage Flavours
    * Manage Nova user keypairs
    * Manage Projet Security Groups
    * Manage Nova compute servers
    * Manage Quota
    * Get Nova Stats (host, services, tanents)
    * Verify operation of the Compute Service
2. __Advanced__ 
    * Manage Compute Instances action (e.g., Launch, Shutdown, terminate, etc.)
    * Manage Folating IP (e.g., Configuration, Assignment, Detach)
    * Configure Security Groups 
    * Manage Nova host console (RDP, Splice, tty)
    * Access instances securely using key-pair
    * Manage instance snapshots

# 1. Introduction to Compute Engine





## Topics 
1. __Compute Service Architecture:__ Understanding the overall service based model. 
2. __Implementation of Compute Hosts in the User Plane:__ Implementaion with several hypervior back-ends (baremetals and container)
3. __Managing Flavours:__ template based resource allocation 
4. __Managing Keypairs:__ Securing access to the copute hosts
5. __Security Groups:__ Access-lists and firewalls to control network traffic    
6. __Resource Quotas:__ Controlled provissioning of resources
7. __Manage Compute Hosts:__ play with instances  
8. __Compute Service Monitoring:__ Telemetry  

## 1.1. Nova Compute Architecture
![](figs/nova_1.png)

* __Nova Core components__ 
    * __API__: Communicates with cloud users and other OpenStack services (e.g. Security, Networking, Images and Volume). 
    * __Conductor__: Intermediatory between Compute hosts and Nova Database  
    * __Schedular__: Decides where to place an instance to run them? 
* __Nova DB__: Store nova states (current state of compute service with all virtual resources and state of the instances).
* __Compute Hosts__ 
    * Executes cloud user instances
    * Download images form glance and maintain them in a cache.  
    * Updates Glance images with snapshots. 

## 1.2. Nova Compute in User Plane 
![](figs/nova_2.png)

* __KVM Mackend__: Nova compute sends instruction to the respective hypervisor backend
* __ESXi Backend__: For ESXi backend the communication takes place between VCenter and Nova. VCenter is responsible for placing the instance into ESXi hosts, Nova is not responsible for plaing in a Host, that is done by the VMWare DRS (Dynamic Resource Scheduler).  
* __Baremetal Backend__: Nova Communicates with Ironic service to provission a Bare metal server. 
* __Container backend__: There are three ways to launch Containers:
    1. __Magnum__: Launch Docker Natively and manage using Docker Swarm.
    2. __LXC__: Provission containers with LXC drivers and treat them as Instances.
    3. __Zun__: Openstack native serice.

## Managing SSH Keypairs
* Secure access to instance shell from external networks. SSH sessions are secured by encrypting the traffic using a _Asymetric Key Crypto System_ e.g., RSA. 
* Private-Public key pair:
    * __Public Key:__ Injected to an instance at boot time and owned by the owner of the instance. 
    * __Private Key:__ Stored in secured file on local file system and used by the `ssh` command to access instance. 
* Keypair action:
    * __Create__: Using openstack or external KeyGen (upload manually). 
    * __Delete__: Delete an existing key
    * __List__: list all public keys
    * __Show__: show details 

## 1.4. Flavours:
* Template base resource provissioning.
* The Flavour class
```
Flavour{
    # Properties 
    name:str,           # name of the flavour
    vcpu:int,           # number of virtual CPUs
    ram:int,            # size of the virtual memory in MB 
    disk:int,           # size of the disk in GB
    ephemeral:int,      # size of the empherical disk in GB
    swap:int,           # size of the swap disk in GB
    rxtx:float,         # Receive and Tranmit factor for XEN instances
    is_public:boot,     # if true, flavour is visible to all projects
    extra_specs:dict    # metadata, cpu limit, memory limit, etc. 

    # Methods 
    create() {...}      # create a flavour
    delete() {...}      # delete a flavour
    list() {...}        # list all flavours 
    show() {...}        # show details
    set() {...}         # set a property      
    
}
```



# Lab Workbook


## Task 1. Project and User creation 
1. Create a project `FIT`
2. Create Two users `fit_admin/Password123..` and `fit_user/Password123..`
3. Assign the users to the project
4. Verigy login with a scope `fit_admin:FIT` and `fit_user:FIT`

### Steps:
1. Login as `admin/openstack`
2. Identity -> Projects -> Create Project
    ```
    {
        name: FIT,
        enabled: True
    }
    ```
3. Identity -> Users -> Create User
    * Projct admin
    ```
    {
        user_name : fit_admin,
        password: Password123..,
        primary_project: FIT,
        Role: admin,
        Enabled: True
    }
    ```
    * Project user
    ```
    {
        user_name : fit_user,
        password: Password123..,
        primary_project: FIT,
        Role: __member__
        Enabled: True
    }
    ```
3. Sign out form the `admin` account 
4. Login back as `fit_admin/Password123.. : FIT`


## Task 2: Deploy an Ubuntu Cloud Image 
1. Download an ubuntu cloud image from: https://cloud-images.ubuntu.com/focal/
2. Unlock the root user of the image.


### Steps:
1. Download image from: https://cloud-images.ubuntu.com/focal/
2. Upload the file into COA VM
3. run the following script 
    ```
    image_name=focal-server-cloudimg-amd64.img
    passwd=Password123..
    sudo apt -y update 
    sudo apt -y install libguestfs-tools
    virt-customize -a $image_name --root-password password:$passwd
    ```